In [2]:
import pyspark
from pyspark.sql import SparkSession

In [106]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, unix_timestamp

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-03 22:13:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Распознаётся d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)… 2600:9000:244f:1000:b:20a5:b140:21, 2600:9000:244f:c600:b:20a5:b140:21, 2600:9000:244f:2800:b:20a5:b140:21, ...
Подключение к d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:244f:1000:b:20a5:b140:21|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 64346071 (61M) [binary/octet-stream]
Сохранение в: «yellow_tripdata_2024-10.parquet»

yellow_tripdata_202 100%[===================>]  61,36M  22,7MB/s    за 2,7s    

2025-03-03 22:13:07 (22,7 MB/s) - «yellow_tripdata_2024-10.parquet» сохранён [64346071/64346071]



## Question 1: Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

In [7]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('DE Zoomcamp Hw 5') \
    .getOrCreate()

25/03/03 22:14:58 WARN Utils: Your hostname, Ainas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.178.218 instead (on interface en0)
25/03/03 22:14:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 22:14:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark.version

'3.5.5'

## Question 2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 6MB
- **25MB**
- 75MB
- 100MB

In [12]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [16]:
df.show(2)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [19]:
df.repartition(4).write.parquet('yellow_tripdata_2024-10_repartitioned/') 

In [23]:
!ls -lh yellow_tripdata_2024-10_repartitioned/

total 183496
-rw-r--r--  1 palma  staff     0B  3 мар 22:20 _SUCCESS
-rw-r--r--  1 palma  staff    22M  3 мар 22:20 part-00000-d1bcf478-902e-4b0d-afc1-51784e641e95-c000.snappy.parquet
-rw-r--r--  1 palma  staff    22M  3 мар 22:20 part-00001-d1bcf478-902e-4b0d-afc1-51784e641e95-c000.snappy.parquet
-rw-r--r--  1 palma  staff    22M  3 мар 22:20 part-00002-d1bcf478-902e-4b0d-afc1-51784e641e95-c000.snappy.parquet
-rw-r--r--  1 palma  staff    22M  3 мар 22:20 part-00003-d1bcf478-902e-4b0d-afc1-51784e641e95-c000.snappy.parquet


## Question 3: Count records

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 85,567
- 105,567
- **125,567**
- 145,567

In [28]:
df \
.filter('date(tpep_pickup_datetime) = date("2024-10-15")') \
.count()

128893

## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

- 122
- 142
- **162**
- 182

In [107]:
df = df \
.filter('date(tpep_pickup_datetime) >= date("2024-10-01") and date(tpep_dropoff_datetime) <= date("2024-10-31")') \
.withColumn('trip_duration', col('tpep_dropoff_datetime') - col('tpep_pickup_datetime')) \
.withColumn("diff_in_hours", (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 3600)

In [118]:
df.agg(
    F.max('trip_duration').alias('max_trip_duration'),
    F.max('diff_in_hours').cast('int').alias('max_diff_in_hours')
).show(1,False)

+-----------------------------------+-----------------+
|max_trip_duration                  |max_diff_in_hours|
+-----------------------------------+-----------------+
|INTERVAL '6 18:37:04' DAY TO SECOND|162              |
+-----------------------------------+-----------------+



In [119]:
df.select(
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'trip_duration',
    'diff_in_hours'
) \
.filter(col('trip_duration').cast('string') == lit("INTERVAL '6 18:37:04' DAY TO SECOND")) \
.show(1,False)

+--------------------+---------------------+-----------------------------------+------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_duration                      |diff_in_hours     |
+--------------------+---------------------+-----------------------------------+------------------+
|2024-10-16 13:03:49 |2024-10-23 07:40:53  |INTERVAL '6 18:37:04' DAY TO SECOND|162.61777777777777|
+--------------------+---------------------+-----------------------------------+------------------+



## Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- **4040**
- 8080

In [ ]:
http://localhost:4040/jobs/

## Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

- **Governor's Island/Ellis Island/Liberty Island**
- Arden Heights
- Rikers Island
- Jamaica Bay

In [84]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-03 23:08:20--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Распознаётся d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)… 2600:9000:244f:7200:b:20a5:b140:21, 2600:9000:244f:2800:b:20a5:b140:21, 2600:9000:244f:9e00:b:20a5:b140:21, ...
Подключение к d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:244f:7200:b:20a5:b140:21|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 12331 (12K) [text/csv]
Сохранение в: «taxi_zone_lookup.csv»

taxi_zone_lookup.cs 100%[===================>]  12,04K  --.-KB/s    за 0s      

2025-03-03 23:08:20 (3,83 GB/s) - «taxi_zone_lookup.csv» сохранён [12331/12331]



In [125]:
df_taxi_zone_lookup = spark.read.option('header', 'true').csv('taxi_zone_lookup.csv')

In [126]:
df_taxi_zone_lookup.createOrReplaceTempView('taxi_zone_lookup')

In [129]:
spark.sql('''
    select *
    from taxi_zone_lookup
''').show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [128]:
df.createOrReplaceTempView('yellow_taxi')

In [138]:
spark.sql('''
    select
        Zone,
        count(*) as trips_cnt
    from yellow_taxi yt
    left join taxi_zone_lookup tzl
    where yt.PULocationID = tzl.LocationID
    group by Zone
    order by count(*)
''').show(5, False)

+---------------------------------------------+---------+
|Zone                                         |trips_cnt|
+---------------------------------------------+---------+
|Governor's Island/Ellis Island/Liberty Island|1        |
|Rikers Island                                |2        |
|Arden Heights                                |2        |
|Jamaica Bay                                  |3        |
|Green-Wood Cemetery                          |3        |
+---------------------------------------------+---------+
only showing top 5 rows

